# Machine Learning in Julia

## A neural network in 10 lines of Julia code

Let's define a fully-connected two layer network.

In [ ]:
dense(W, b, σ = identity) = x -> σ.(W * x .+ b)

chain(f...) = foldl(∘, reverse(f))

m = chain(
    dense(randn(5,10), randn(5), tanh),
    dense(randn(2,5), randn(2)))

In [ ]:
x = rand(10); # some input

In [ ]:
m(x)

Let's train it!

In [ ]:
using Zygote # source-to-source reverse mode AD

dm, = gradient(m) do model
    sum(model(x))
end

In [ ]:
m.f.W

In [ ]:
dm.f.W

In [ ]:
η = 0.01

m.f.W .-= η * dm.f.W # Gradient descent!

## Flux - The ML library that doesn't make you tensor

Web page: https://fluxml.ai/, Examples: [Model zoo](https://github.com/FluxML/model-zoo/)

<img src="https://fluxml.ai/logo.png" width=300>

<img src="flux.png" width=800>

In [ ]:
using Flux

In [ ]:
m = Chain(
    Dense(10, 5),
    Dense(5, 2),
    softmax # normalize output neurons
)

In [ ]:
data, labels = rand(10, 100), fill(0.5, 2, 100); # fake data

In [ ]:
loss(x, y) = sum(Flux.mse(m(x), y)) # mean squared error

In [ ]:
opt = Descent(0.01) # or ADAM

In [ ]:
Flux.train!(loss, params(m), [(data,labels)], opt)

In [ ]:
m(rand(10)) # trained model